In [1]:
%matplotlib inline 

import math
import copy
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch import optim, nn

import model
import model_utils
import preprocessing_utils

import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString
import contextily as ctx
import matplotlib.pyplot as plt

from os.path import dirname, abspath
import os
import sys

In [2]:
torch.manual_seed(1234)

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # Heroku only have cpu


num_timesteps_input = 7 # Default is 30 minutes
num_timesteps_output = 2 # Default is 10 minutes

loss_criterion = nn.MSELoss()

In [4]:
raw_dir = os.path.join(os.getcwd(), 'data', 'raw')
process_dir = os.path.join(os.getcwd(), 'data', 'processed')

# overwrite = False means that the processing function will only run if the process data files do not exist
# overwrite = True => functions will run regardless
preprocessing_utils.processed(raw_dir, process_dir, overwrite=False)
A, X, metadata, cat2index, timesteps, means, stds = preprocessing_utils.load(process_dir)


test_original_data = X


test_input, test_target = preprocessing_utils.generate_dataset(test_original_data,
                                           num_timesteps_input=num_timesteps_input,
                                           num_timesteps_output=num_timesteps_output)

# input shape (num_samples ,num_vertices, num_timesteps_window, num_features)
# output shape (num_samples ,num_vertices, num_timesteps_window)
adj_mat = preprocessing_utils.get_normalized_adj(A)
adj_mat = torch.from_numpy(adj_mat).to(device)

In [5]:
X.shape

(53, 3, 7)

In [6]:
i = 0
indices = [(i, i + (num_timesteps_input + num_timesteps_output))]

# Save samples
features, target = [], []
for i, j in indices:
    features.append(
        X[:, :, i: i + num_timesteps_input].transpose(
            (0, 2, 1)))
    target.append(X[:, 0, i + num_timesteps_input: j])

In [7]:
test_input = torch.from_numpy(np.array(features)).to(device)

In [8]:
test_target = torch.from_numpy(np.array(target)).to(device)

In [9]:
# Load model
saved_models_path = os.path.join(dirname(os.getcwd()), 'saved_models', 'last_saved_model.txt')
with open(saved_models_path) as f:
    saved_model = f.read()

latest_model_path = os.path.join(dirname(os.getcwd()), saved_model)
checkpoint = torch.load(latest_model_path, map_location=None)
model_stgcn = model.Stgcn_Model(checkpoint['model_nodes_num'], checkpoint['model_features_num'], checkpoint['model_input_timesteps'], checkpoint['model_num_output'])
model_stgcn.load_state_dict(checkpoint['state_dict'])
optimizer = optim.Adam(model_stgcn.parameters(), lr=checkpoint['model_lr'])
optimizer = optimizer.load_state_dict(checkpoint['opti_state_dict'])

In [10]:
loaded_model = model_stgcn
loaded_model.to(device)
loaded_optimizer = optimizer

In [11]:
test_input.shape

torch.Size([1, 53, 7, 3])

In [12]:
test_target.shape

torch.Size([1, 53, 0])

In [13]:
test_input.is_cuda

True

In [14]:
adj_mat.is_cuda

True

In [15]:
torch.manual_seed(1234)


predicted = model_utils.predict(loaded_model, test_input, adj_mat)
predicted = predicted.cpu().numpy()
predicted_denorm = preprocessing_utils.denormalize(predicted, stds[0], means[0])

In [16]:
predicted

array([[[-1.0906575 , -1.0229636 , -0.930424  , -0.8908162 ],
        [ 0.34363836,  0.19998197,  0.12976894,  0.09501595],
        [-0.6031116 , -0.41707903, -0.24092524, -0.07884623],
        [ 0.3337333 ,  0.23605508,  0.18649082,  0.14834271],
        [ 0.91212976,  0.9326386 ,  0.9227997 ,  0.9147212 ],
        [-0.4083112 , -0.50697845, -0.5739235 , -0.5818916 ],
        [ 0.8408299 ,  0.7397768 ,  0.6883941 ,  0.6626566 ],
        [-0.720857  , -0.66603637, -0.62737334, -0.59841967],
        [ 0.7964021 ,  0.84528226,  0.8412879 ,  0.83432806],
        [-0.62074614, -0.6284614 , -0.6223684 , -0.6137146 ],
        [ 0.35565573,  0.32931775,  0.3089708 ,  0.30421036],
        [-0.88826597, -0.9533316 , -0.9888438 , -0.9855536 ],
        [-0.3669536 , -0.14249368,  0.01493241,  0.07078905],
        [ 0.78339887,  0.6583471 ,  0.58531713,  0.50920355],
        [-0.16910355, -0.20736112, -0.20665021, -0.21880741],
        [ 0.3601808 ,  0.31514403,  0.29237378,  0.2675956 ],
        

In [17]:
predicted_denorm

array([[[2.7098317, 2.758624 , 2.825325 , 2.8538735],
        [3.743645 , 3.6401   , 3.5894918, 3.5644426],
        [3.0612454, 3.1953342, 3.3223026, 3.439126 ],
        [3.7365055, 3.666101 , 3.6303759, 3.6028795],
        [4.153403 , 4.168185 , 4.1610937, 4.1552706],
        [3.201654 , 3.1305363, 3.0822837, 3.0765405],
        [4.102011 , 4.029174 , 3.9921384, 3.9735873],
        [2.9763768, 3.0158906, 3.0437582, 3.0646274],
        [4.0699883, 4.1052203, 4.102341 , 4.097325 ],
        [3.0485349, 3.0429738, 3.0473654, 3.053603 ],
        [3.752307 , 3.7333229, 3.7186573, 3.715226 ],
        [2.8557115, 2.8088136, 2.783217 , 2.7855885],
        [3.2314637, 3.3932502, 3.5067198, 3.5469804],
        [4.060616 , 3.970481 , 3.9178424, 3.8629813],
        [3.3740702, 3.346495 , 3.3470073, 3.3382447],
        [3.7555685, 3.7231069, 3.7066944, 3.6888347],
        [3.2917151, 3.2579978, 3.2381659, 3.2409608],
        [2.1401   , 2.2102613, 2.2802253, 2.3260736],
        [3.0910234, 3.259688

In [19]:
predicted_denorm

array([[[2.7098317, 2.758624 , 2.825325 , 2.8538735],
        [3.743645 , 3.6401   , 3.5894918, 3.5644426],
        [3.0612454, 3.1953342, 3.3223026, 3.439126 ],
        [3.7365055, 3.666101 , 3.6303759, 3.6028795],
        [4.153403 , 4.168185 , 4.1610937, 4.1552706],
        [3.201654 , 3.1305363, 3.0822837, 3.0765405],
        [4.102011 , 4.029174 , 3.9921384, 3.9735873],
        [2.9763768, 3.0158906, 3.0437582, 3.0646274],
        [4.0699883, 4.1052203, 4.102341 , 4.097325 ],
        [3.0485349, 3.0429738, 3.0473654, 3.053603 ],
        [3.752307 , 3.7333229, 3.7186573, 3.715226 ],
        [2.8557115, 2.8088136, 2.783217 , 2.7855885],
        [3.2314637, 3.3932502, 3.5067198, 3.5469804],
        [4.060616 , 3.970481 , 3.9178424, 3.8629813],
        [3.3740702, 3.346495 , 3.3470073, 3.3382447],
        [3.7555685, 3.7231069, 3.7066944, 3.6888347],
        [3.2917151, 3.2579978, 3.2381659, 3.2409608],
        [2.1401   , 2.2102613, 2.2802253, 2.3260736],
        [3.0910234, 3.259688

In [20]:
predicted_denorm[:,:, :2]

array([[[2.7098317, 2.758624 ],
        [3.743645 , 3.6401   ],
        [3.0612454, 3.1953342],
        [3.7365055, 3.666101 ],
        [4.153403 , 4.168185 ],
        [3.201654 , 3.1305363],
        [4.102011 , 4.029174 ],
        [2.9763768, 3.0158906],
        [4.0699883, 4.1052203],
        [3.0485349, 3.0429738],
        [3.752307 , 3.7333229],
        [2.8557115, 2.8088136],
        [3.2314637, 3.3932502],
        [4.060616 , 3.970481 ],
        [3.3740702, 3.346495 ],
        [3.7555685, 3.7231069],
        [3.2917151, 3.2579978],
        [2.1401   , 2.2102613],
        [3.0910234, 3.2596884],
        [3.648703 , 3.7163389],
        [3.3023143, 3.2764468],
        [3.5489397, 3.4985967],
        [3.0143275, 3.0512953],
        [3.4622226, 3.4067256],
        [3.890124 , 3.8543527],
        [3.744612 , 3.7435532],
        [4.074223 , 3.9704607],
        [3.557597 , 3.5099294],
        [3.5422184, 3.5000637],
        [3.4325845, 3.384962 ],
        [3.9242637, 3.961522 ],
        

In [21]:
predicted_denorm.shape

(1, 53, 4)

In [22]:
timestep_speedbands = predicted_denorm.reshape(predicted_denorm.shape[2], predicted_denorm.shape[1])

In [23]:
timestep_speedbands

array([[2.7098317, 2.758624 , 2.825325 , 2.8538735, 3.743645 , 3.6401   ,
        3.5894918, 3.5644426, 3.0612454, 3.1953342, 3.3223026, 3.439126 ,
        3.7365055, 3.666101 , 3.6303759, 3.6028795, 4.153403 , 4.168185 ,
        4.1610937, 4.1552706, 3.201654 , 3.1305363, 3.0822837, 3.0765405,
        4.102011 , 4.029174 , 3.9921384, 3.9735873, 2.9763768, 3.0158906,
        3.0437582, 3.0646274, 4.0699883, 4.1052203, 4.102341 , 4.097325 ,
        3.0485349, 3.0429738, 3.0473654, 3.053603 , 3.752307 , 3.7333229,
        3.7186573, 3.715226 , 2.8557115, 2.8088136, 2.783217 , 2.7855885,
        3.2314637, 3.3932502, 3.5067198, 3.5469804, 4.060616 ],
       [3.970481 , 3.9178424, 3.8629813, 3.3740702, 3.346495 , 3.3470073,
        3.3382447, 3.7555685, 3.7231069, 3.7066944, 3.6888347, 3.2917151,
        3.2579978, 3.2381659, 3.2409608, 2.1401   , 2.2102613, 2.2802253,
        2.3260736, 3.0910234, 3.2596884, 3.381306 , 3.4360013, 3.648703 ,
        3.7163389, 3.734406 , 3.7535872, 3.30231

In [24]:
def loc_to_linestring(loc):
    coordArr = loc.split()
    coordArr = [float(coord) for coord in coordArr]
    return LineString([coordArr[1::-1], coordArr[3:1:-1]])

def plotGeoPerformance(metadata, speedbands):
    df = pd.DataFrame(metadata).transpose()
    df["speedbands"] = speedbands
    loc = df["start_pos"] + " " + df["end_pos"]
    linestrings = loc.apply(loc_to_linestring)
    gdf = gpd.GeoDataFrame(df, geometry=linestrings, crs="EPSG:4326")
    gdf = gdf.to_crs('EPSG:3857')
    fig, ax = plt.subplots(figsize=(10, 10))
    gdf.plot(ax=ax, column="speedbands", legend=True, cmap="OrRd", legend_kwds={'label': 'speedbands'})
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    ctx.add_basemap(ax)

In [25]:
plotGeoPerformance(metadata, timestep_speedbands[0])

CRSError: Invalid projection: EPSG:4326: (Internal Proj Error: proj_create: SQLite error on SELECT name, type, coordinate_system_auth_name, coordinate_system_code, datum_auth_name, datum_code, area_of_use_auth_name, area_of_use_code, text_definition, deprecated FROM geodetic_crs WHERE auth_name = ? AND code = ?: no such column: area_of_use_auth_name)